In [ ]:
"""
Created on Tue Apr 05 15:43 2022

Try using xarray input into the DNN directly

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt

import cartopy
import cartopy.crs as ccrs

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'OPM016'
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
outputpath_nn = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/data/interim/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'
inputpath_tides = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/TIDES/'


FOR EACH POINT:
- T and S extrapolated to ice draft depth
- Distance to front
- Distance to the grounding line
- x-slope ice draft
- y-slope ice draft
- x-slope bedrock
- y-slope bedrock
- Ice draft depth
- Bathymetry
- utide
- Orientation of the cavity
- Ice draft concentration
- Mean bathymetry at entry (to add in future)
- Max bathymetry (to add in future)
- Target: melt m ice per yr

In [ ]:
T_S_2D_isfdraft = xr.open_mfdataset(inputpath_profiles+'T_S_2D_fields_isf_draft.nc').sel(profile_domain=50)

In [ ]:
# dIF, dGL, longitude, latitude
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
# T and S profiles
file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc')
file_TS = file_TS_orig.sel(Nisf=file_isf.Nisf)
file_TS_dom = file_TS.sel(profile_domain=50)

In [ ]:
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics.nc')
# Local slope
local_ice_slope = plume_charac['alpha'].sel(option='appenB').drop('option')

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
# bathymetry, ice draft, concentration
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft'] 
file_isf_conc = file_conc_cut['isfdraft_conc']

In [ ]:
file_slope = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft.nc')
file_orientation = xr.open_dataset(inputpath_mask+'nemo_5km_orientation_info.nc')

In [ ]:
utide_file = xr.open_dataset(inputpath_tides + 'tidal_velocity_nemo_Ant_stereo.nc')
u_tide = dfmt.cut_domain_stereo(utide_file['ttv'], map_lim, map_lim)

In [ ]:
NEMO_melt_rates_2D = xr.open_mfdataset(outputpath_melt+'melt_rates_2D_NEMO.nc')
melt_rate = NEMO_melt_rates_2D['melt_m_ice_per_y']

Collect all 2D data in one dataset

In [ ]:
#, 'longitude', 'latitude'
geometry_2D = file_isf[['dGL', 'dIF']].merge(file_draft).merge(file_bed_orig).merge(file_slope).merge(file_orientation).merge(u_tide)
geometry_2D['dIF'] = geometry_2D['dIF'].where(np.isfinite(geometry_2D['dIF']), np.nan)
time_dpdt_in = T_S_2D_isfdraft[['theta_in','salinity_in']].merge(melt_rate)

In [ ]:
geometry_2D_br, time_dpdt_in_br = xr.broadcast(geometry_2D,time_dpdt_in)

In [ ]:
final_input_xr = xr.merge([geometry_2D_br, time_dpdt_in_br]).transpose('y','x','time').drop('profile_domain')

PREPARE DATAFRAME WITH ALL DATA

In [ ]:
merged_df = final_input_xr.drop('longitude').drop('latitude').to_dataframe()

# remove rows where there are nans
clean_df_yy = merged_df.dropna()



In [ ]:
# remove index (time, x, y)
clean_df_yy.reset_index(drop=True, inplace=True)
clean_df_yy

In [ ]:
new_timetag = True
if new_timetag:
    datetag_dt = datetime.datetime.today()
    timetag_dt = datetime.datetime.now()
    timetag = str(datetag_dt.year)+str(datetag_dt.month).zfill(2)+str(datetag_dt.day).zfill(2)+'-'+str(timetag_dt.hour).zfill(2)+str(timetag_dt.minute).zfill(2)

In [ ]:
clean_df_yy.to_csv(outputpath_nn + 'dataframe_input_'+timetag+'.csv')

DIVIDE INTO TRAIN AND TEST DATASET

In [ ]:
clean_df = pd.read_csv(outputpath_nn + 'dataframe_input_'+timetag+'.csv').drop('Unnamed: 0', 1)

In [ ]:
data_train = clean_df.sample(frac=0.7, axis=0) 
data_test  = clean_df.drop(data_train.index)

In [ ]:
y_train = data_train['melt_m_ice_per_y']
x_train = data_train.drop(['melt_m_ice_per_y'], axis=1)

y_test = data_test['melt_m_ice_per_y']
x_test = data_test.drop(['melt_m_ice_per_y'], axis=1)

print('Original data shape was : ',clean_df.shape)
print('x_train : ',x_train.shape, 'y_train : ',y_train.shape)
print('x_test  : ',x_test.shape,  'y_test  : ',y_test.shape)

### 3.2 - Data normalization
**Note :** 
 - All input data must be normalized, train and test.  
 - To do this we will **subtract the mean** and **divide by the standard deviation**.  
 - But test data should not be used in any way, even for normalization.  
 - The mean and the standard deviation will therefore only be calculated with the train data.

In [ ]:
x_train.mean()/x_train.std()

In [ ]:
norm_method = 'interquart' #'std', 'interquart', 'minmax'

x_mean = x_train.mean()
y_mean = y_train.mean()

if norm_method == 'std':
    x_range  = x_train.std()
    y_range  = y_train.std()
elif norm_method == 'interquart':
    x_range  = x_train.quantile(0.9) - x_train.quantile(0.1)
    y_range  = y_train.quantile(0.9) - y_train.quantile(0.1)
elif norm_method == 'minmax':
    x_range  = x_train.max() - x_train.min() 
    y_range  = y_train.max() - y_train.min() 
    
x_train_norm = (x_train - x_mean)/x_range
x_test_norm = (x_test - x_mean)/x_range

y_train_norm = (y_train - y_mean)/y_range
y_test_norm = (y_test - y_mean)/y_range


In [ ]:
summary_df = pd.DataFrame()
summary_df['x_mean'] = x_mean
summary_df['x_range'] = x_range
summary_df = summary_df.T 
summary_df['melt_m_ice_per_y'] = [y_mean, y_range]
summary_df.to_csv(outputpath_nn + 'dataframe_norm_training_data_'+timetag+'.csv')

In [ ]:
#display(x_train.describe().style.format("{0:.2f}").set_caption("After normalization :"))
#display(x_train.head(5).style.format("{0:.2f}").set_caption("Few lines of the dataset :"))

x_train_arr, y_train_arr = np.array(x_train_norm), np.array(y_train_norm)
x_test_arr,  y_test_arr  = np.array(x_test_norm),  np.array(y_test_norm)

## Step 4 - Build a model
About informations about : 
 - [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
 - [Activation](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
 - [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
 - [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
def get_model_v1(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n1'))
    model.add(keras.layers.Dense(64, activation='relu', name='Dense_n2'))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n3'))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

def get_model_v2(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

#def get_model_v1(shape):
#    nodes = 256
#    #activ = 'sigmoid'
#   activ = 'relu'   # standard
#    #activ = 'tanh'
#    #activ = 'selu'
#    model = keras.models.Sequential()
#    model.add(keras.layers.Input(shape, name="InputLayer"))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n1'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n2'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n3'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n4'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n5'))
#    model.add(keras.layers.Dense(nodes, activation= activ, name='Dense_n6'))
#    model.add(keras.layers.Dense(1, name='Output'))  
#                                    # nbvect = number of elements of target vector
#
#    model.compile(optimizer = 'rmsprop',
#                  loss      = 'mse',                  # mse, mean quadratic error 
#              metrics   = ['mae', 'mse'] )        # mae =  mean absolute error
#    return model

## Step 5 - Train the model
### 5.1 - Get it

In [ ]:
input_size = len(x_train_arr[0])

In [ ]:
model=get_model_v1( (input_size,) )

model.summary()

### 5.2 - Train it

In [ ]:
epoch_nb = 50
batch_siz = 50

In [ ]:
with open(outputpath_doc+timetag+'.log','w') as file:
    file.write('Timetag: '+timetag+' \n')
    file.write('----- DATA ----- \n')
    file.write('Training data from: '+nemo_run+'\n')
    file.write('Norm method: '+norm_method+'\n')
    file.write('Original data shape was : '+str(clean_df.shape)+'\n')
    file.write('x_train : '+str(x_train.shape)+', y_train : '+str(y_train.shape)+'\n')
    file.write('x_test  : '+str(x_test.shape)+', y_test  : '+str(y_test.shape)+'\n') 
    file.write('Input variables: '+','.join(map(str,x_train_norm.columns))+'\n')
    file.write('\n')
    file.write('----- MODEL ----- \n')
    with redirect_stdout(file):
        model.summary()
    file.write('\n')
    file.write('----- TRAINING ----- \n')
    file.write('Epochs: '+str(epoch_nb)+'\n')
    file.write('Batch size: '+str(batch_siz))

In [ ]:
time_start = time.time()
history = model.fit(x_train_arr,
                    y_train_arr,
                    epochs          = epoch_nb,
                    batch_size      = batch_siz,
                    verbose         = 1,
                    validation_data = (x_test_arr, y_test_arr))
time_end = time.time()

In [ ]:
timelength = time_end - time_start
with open(outputpath_doc+timetag+'.log','a') as file:
    file.write('\n Training time (in s): '+str(timelength))

In [ ]:
model.save(outputpath_nn + 'model_nn_'+timetag+'.h5')

## Step 6 - Evaluate
### 6.1 - Model evaluation
MAE =  Mean Absolute Error (between the labels and predictions)  
A mae equal to 3 represents an average error in prediction of $3k.

In [ ]:
score = model.evaluate(x_test_arr, y_test_arr, verbose=1)

print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / mae       : {:5.4f}'.format(score[1]))
print('x_test / mse       : {:5.4f}'.format(score[2]))

### 6.2 - Training history
What was the best result during our training ?

In [ ]:
df=pd.DataFrame(data=history.history)
display(df)

In [ ]:
print("min( val_mae ) : {:.4f}".format( min(history.history["val_mae"]) ) )

In [ ]:
diag.plot_history(history, plot={'MSE' :['mse', 'val_mse'],
                                'MAE' :['mae', 'val_mae'],
                                'LOSS':['loss','val_loss']})

In [ ]:
plt.close('all')

## Step 7 - Make a prediction
The data must be normalized with the parameters (mean, std) previously used.

In [ ]:
for kk, yy in enumerate(tqdm(range(2007,2008))):

    # take profile of T and S for that year
    TS_isf_tt = time_dpdt_in.sel(time=yy).drop('time')
    # T and S profiles to dataframe
    TS_isf_df = TS_isf_tt.drop('longitude').drop('latitude').to_dataframe()

    # Geometry info to dataframe
    length_df = len(geometry_2D.x)*len(geometry_2D.y)
    geo_df = geometry_2D.drop('longitude').drop('latitude').to_dataframe()

    # write out melt rate for that year (target)
    melt_rate_isf_tt = melt_rate.sel(time=yy)
    # transform to dataframe
    melt_df = melt_rate_isf_tt.to_dataframe()#.drop(['mapping'],axis=1)#.reset_index()
    melt_df = melt_df.drop(['time'],axis=1)

    # merge properties and melt
    merged_df0 = pd.merge(geo_df,melt_df,how='left',on=['x','y'])
    merged_df_val = pd.merge(TS_isf_df,merged_df0,how='left',on=['x','y'])

    # remove rows where there are nans
    clean_df_yy_val = merged_df_val.dropna()
    # remove the x and y axis
    # clean_df = clean_df.drop(['x'], axis=1).drop(['y'], axis=1)
    if kk > 0:
        merged_yy_df_val = merged_yy_df_val.append(clean_df_yy, ignore_index = True)
    else:
        merged_yy_df_val = clean_df_yy_val.copy()    

In [ ]:
clean_df_val = merged_yy_df_val.reset_index().drop(['x'], axis=1).drop(['y'], axis=1)

In [ ]:
y_val = clean_df_val['melt_m_ice_per_y']
x_val = clean_df_val.drop(['melt_m_ice_per_y'], axis=1).drop('profile_domain', 1)

x_val_norm = x_val.copy()

for ccol in ['dGL','dIF','alpha','bedrock_slope','corrected_isfdraft','bathy_metry','theta_in','salinity_in']:
    mean = x_train[ccol].mean()
    std  = x_train[ccol].std()
    x_val_norm[ccol] = (x_val[ccol] - mean) / std

x_val_arr, y_val_arr = np.array(x_val_norm), np.array(y_val)

#my_data=np.array(x_val_arr)#.reshape(1,13)

In [ ]:
x_val_arr.shape

In [ ]:
x_val_norm

In [ ]:
y_out = model.predict(x_val_arr)

In [ ]:
y_out

In [ ]:
y_out_pd_s = pd.Series(y_out[:,0],index=merged_yy_df_val.index,name='computed_melt') 
y_target_pd_s = pd.Series(y_val_arr,index=merged_yy_df_val.index,name='reference_melt') 

In [ ]:
y_out_xr = y_out_pd_s.to_xarray()
y_target_xr = y_target_pd_s.to_xarray()
y_to_compare = xr.merge([y_out_xr.T, y_target_xr.T]).sortby('y')

In [ ]:
xx = range(0,80)
plt.figure()
plt.scatter(y_to_compare['computed_melt'].values.flatten(),y_to_compare['reference_melt'].values.flatten(), s=10, edgecolors='None',alpha=0.2)
plt.plot(xx,xx,'k')

In [ ]:
computed_melt = y_to_compare['computed_melt']
ref_melt = y_to_compare['reference_melt']

In [ ]:
min_m = min(computed_melt.min(), ref_melt.min())
max_m = max(computed_melt.max(), ref_melt.max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
else:
    cmap = mpl.cm.viridis
    minlim = 0

f = plt.figure(figsize=(15, 5))

ax1 = plt.subplot(1, 3, 1)
computed_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=cmap)
ax1.set_title('Neural Network [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
ref_melt.plot(ax=ax2, vmin=minlim,vmax=lim, cmap=cmap)
ax2.set_title('Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
(computed_melt - ref_melt).plot(ax=ax3)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('NN - Ref [m ice/y]')

f.tight_layout()

In [ ]:
y_to_compare.to_netcdf(outputpath_nn+'prediction_linreg_2007_allisf.nc')

In [ ]:
rmse = np.sqrt(np.mean((y_out - y_val_arr)**2))

In [ ]:
rmse

In [ ]:
predictions = model.predict( my_data )
print("Prediction : {:.2f} m ice per y".format(predictions[0][0]))
print("Reality    : {:.2f} m ice per y".format(y_val_arr[0]))

COMPARE WITH SIMPLE

In [ ]:
sns.set_context('paper')

In [ ]:
kisf = 44
mask_isf = file_isf['ISF_mask']==kisf
if kisf == 11 or kisf == 21:
    mask_isf = (file_isf['ISF_mask']==11) | (file_isf['ISF_mask']==21)

In [ ]:
simple_param = xr.open_dataset(outputpath_melt+'melt_rates_2D_quadratic_local_tuned_correctedTS.nc')
melt_simple = simple_param['melt_m_ice_per_y'].sel(profile_domain=50,time=2007).where(mask_isf, drop=True)

In [ ]:
y_to_compare_nn = xr.open_dataset(outputpath_nn+'prediction_linreg_2007_allisf.nc')
computed_melt = y_to_compare_nn['computed_melt'].where(mask_isf, drop=True)
ref_melt = y_to_compare_nn['reference_melt'].where(mask_isf, drop=True)

In [ ]:
ref_melt.max()

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'
min_m = min(computed_melt.min(),ref_melt.min())
max_m = max(computed_melt.max(),ref_melt.max())
#min_m = min((computed_melt - ref_melt).min(),(melt_simple - ref_melt).min())
#max_m = max((computed_melt - ref_melt).max(),(melt_simple - ref_melt).max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
else:
    cmap = mpl.cm.viridis
    minlim = 0

f = plt.figure()
f.set_size_inches(8.24*1.3, 8.24/3)

ax1 = plt.subplot(1, 3, 1)
ref_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=mpl.cm.coolwarm)
ax1.set_title('Reference [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=minlim,vmax=lim, cmap=mpl.cm.BrBG_r)
#(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=-5,vmax=5, cmap=mpl.cm.BrBG_r)
ax2.set_title('Neural Network param - Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
(melt_simple - ref_melt).plot(ax=ax3, vmin=minlim,vmax=lim, cmap=mpl.cm.BrBG_r)
#(melt_simple - ref_melt).plot(ax=ax3, vmin=-5,vmax=5, cmap=mpl.cm.BrBG_r)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('Simple physical param - Reference [m ice/y]')

f.tight_layout()
f.savefig(plot_path+'comparison_proof_of_concept_nn_antarctica_isf'+str(kisf).zfill(3)+'.png', dpi=300)

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'
min_m = min(computed_melt.min(),ref_melt.min())
max_m = max(computed_melt.max(),ref_melt.max())
#min_m = min((computed_melt - ref_melt).min(),(melt_simple - ref_melt).min())
#max_m = max((computed_melt - ref_melt).max(),(melt_simple - ref_melt).max())
lim = max(abs(min_m),abs(max_m))

if min_m < 0:
    cmap = mpl.cm.coolwarm
    minlim = -lim
else:
    cmap = mpl.cm.viridis
    minlim = 0

f = plt.figure()
f.set_size_inches(8.24*1.3, 8.24/3)

ax1 = plt.subplot(1, 3, 1)
ref_melt.plot(ax=ax1, vmin=minlim,vmax=lim, cmap=mpl.cm.coolwarm)
ax1.set_title('Reference [m ice/y]')

ax2 = plt.subplot(1, 3, 2, sharex = ax1, sharey = ax1)
#(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=minlim,vmax=lim, cmap=mpl.cm.BrBG_r)
(y_to_compare_nn['computed_melt'] - ref_melt).plot(ax=ax2, vmin=-5,vmax=5, cmap=mpl.cm.BrBG_r)
ax2.set_title('Neural Network param - Reference [m ice/y]')

ax3 = plt.subplot(1, 3, 3, sharex = ax1, sharey = ax1)
#(melt_simple - ref_melt).plot(ax=ax3, vmin=minlim,vmax=lim, cmap=mpl.cm.BrBG_r)
(melt_simple - ref_melt).plot(ax=ax3, vmin=-5,vmax=5, cmap=mpl.cm.BrBG_r)
ax3.set_xticklabels('')
ax3.set_yticklabels('')
ax3.set_title('Simple physical param - Reference [m ice/y]')

f.tight_layout()
f.savefig(plot_path+'comparison_proof_of_concept_nn_antarctica_isf'+str(kisf).zfill(3)+'_minmax5.png', dpi=300)

In [ ]:
#ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0,true_scale_latitude=-71))
#ax.coastlines(resolution='50m', linewidth=0.5)
#ax.pcolormesh(melt_simple.longitude,melt_simple.latitude,melt_simple,transform=ccrs.PlateCarree(),rasterized=True)
#ax.set_extent([-180, 180, -90, -60], crs=ccrs.PlateCarree())

======== TO KEEP FOR THE FUTURE =========

In [ ]:
# For each column - for normalization with min and max

normalized_clean_df = clean_df.copy()

for ccol in ['dGL','dIF','alpha','bedrock_slope','corrected_isfdraft','bathy_metry','longitude','latitude','melt_cavity','time']:
    max_ccol = clean_df[ccol].max()
    min_ccol = clean_df[ccol].min()
    normalized_clean_df[ccol] = (clean_df[ccol] - min_ccol)/(max_ccol - min_ccol)

max_T = clean_df[T_list].max().max()
min_T = clean_df[T_list].min().min()
max_S = clean_df[S_list].max().max()
min_S = clean_df[S_list].min().min()

for ccol in [T_list]:
    normalized_clean_df[ccol] = (clean_df[ccol] - min_T)/(max_T - min_T)

for ccol in [S_list]:
    normalized_clean_df[ccol] = (clean_df[ccol] - min_S)/(max_S - min_S)